In [6]:
import cnfg
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
from instagram.client import InstagramAPI 

In [8]:
from db_setup import Base, User, Media, Follower
from models import get_user_id

In [5]:
engine = create_engine('sqlite:///instagram.sqlite')
Base.metadata.bind = engine
DBSession = sessionmaker(bind = engine)
session = DBSession()

In [7]:
config = cnfg.load(".instagram_config")
api = InstagramAPI(client_id=config['CLIENT_ID'],client_secret=config['CLIENT_SECRET'])

In [11]:
key_id = get_user_id('jstnstwrt')

In [ ]:
list_of_followers = session.query(Follower).filter_by(user_id=key_id)